In [1]:
import openai
import json
import os
from IPython.display import display, HTML, JSON, Markdown
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

#### Function definition for the model

In [3]:
def get_function_call(messages, function_call="auto"):
    # Define the functions to use
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = openai.ChatCompletion.create(
        deployment_id="gpt-35-turbo-0613",
        messages=messages,
        functions=functions,
        function_call=function_call,
    )

    return response

In [13]:
first_message = [
    {"role": "user", "content": "What's the weather like in San Francisco?"}]
# 'auto' : Let the model decide what function to call
print("Let the model decide what function to call:")
#print(get_function_call(first_message, "auto"))
function_to_call_response = get_function_call(first_message, "auto")
function_name = function_to_call_response["choices"][0]["message"]["function_call"]["name"]
function_parameters = json.loads( function_to_call_response["choices"][0]["message"]["function_call"]["arguments"])
print (f""" \nfunction: {function_name} with parameters: {function_parameters}""")


Let the model decide what function to call:
 
function: get_current_weather with parameters: {'location': 'San Francisco, CA'}


### Calling multiple functions

#### Define functions

In [14]:
import pytz
from datetime import datetime


def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [15]:
get_current_time("America/New_York")

'04:13:15 AM'

In [19]:
import math


def calculator(num1, num2, operator):
    if operator == '+':
        return str(num1 + num2)
    elif operator == '-':
        return str(num1 - num2)
    elif operator == '*':
        return str(num1 * num2)
    elif operator == '/':
        return str(num1 / num2)
    elif operator == '**':
        return str(num1 ** num2)
    elif operator == 'sqrt':
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [20]:
print(calculator(5, 5, '+'))

10


#### Define functions prompt

In [21]:
functions = [
    {
        "name": "get_current_time",
        "description": "Get the current time in a given location",
        "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                    }
                },
            "required": ["location"],
        },
    },
    {
        "name": "calculator",
        "description": "A simple calculator used to perform basic arithmetic operations",
        "parameters": {
                "type": "object",
                "properties": {
                    "num1": {"type": "number"},
                    "num2": {"type": "number"},
                    "operator": {"type": "string", "enum": ["+", "-", "*", "/", "**", "sqrt"]},
                },
            "required": ["num1", "num2", "operator"],
        },
    }
]

available_functions = {
    "get_current_time": get_current_time,
    "calculator": calculator,
}

In [25]:
import inspect

# helper method used to check if the correct arguments are provided to a function

def check_args(function, args):
    sig = inspect.signature(function)
    print (f""" function signature {sig}\n""")
    params = sig.parameters
    print (f""" function parameters {params}\n""")

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

#### Put all phases together

In [26]:
def run_conversation(messages, functions, available_functions, deployment_id):
    
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message["function_call"]["name"]

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(
            response_message["function_call"]["arguments"])
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

In [27]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(
    messages, functions, available_functions, "gpt-35-turbo-0613")
print(assistant_response['choices'][0]['message'])

Recommended Function call:
{
  "name": "get_current_time",
  "arguments": "{\n  \"location\": \"America/New_York\"\n}"
}

 function signature (location)

 function parameters OrderedDict([('location', <Parameter "location">)])

Output of function call:
05:06:43 AM

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'name': 'get_current_time', 'content': '{\n  "location": "America/New_York"\n}'}
{'role': 'function', 'name': 'get_current_time', 'content': '05:06:43 AM'}

{
  "role": "assistant",
  "content": "It is currently 5:06 AM in New York."
}


In [29]:
messages = [
    {"role": "user", "content": "Last month Fabrikam made $73,846 in sales. Based on that, what would the annual run rate be?"}
]

assistant_response = run_conversation(
    messages, functions, available_functions, "gpt-35-turbo-0613")
print(assistant_response['choices'][0]['message'])

Recommended Function call:
{
  "name": "calculator",
  "arguments": "{\n  \"num1\": 73846,\n  \"num2\": 12,\n  \"operator\": \"*\"\n}"
}

 function signature (num1, num2, operator)

 function parameters OrderedDict([('num1', <Parameter "num1">), ('num2', <Parameter "num2">), ('operator', <Parameter "operator">)])

Output of function call:
886152

Messages in second request:
{'role': 'user', 'content': 'Last month Fabrikam made $73,846 in sales. Based on that, what would the annual run rate be?'}
{'role': 'assistant', 'name': 'calculator', 'content': '{\n  "num1": 73846,\n  "num2": 12,\n  "operator": "*"\n}'}
{'role': 'function', 'name': 'calculator', 'content': '886152'}

{
  "role": "assistant",
  "content": "The annual run rate for Fabrikam based on last month's sales of $73,846 would be $886,152."
}
